<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: 
* Reward function: 
* State: 
* Environment: 

<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages

In [73]:
!pip install -r requirements.txt --user

  Using cached scikit_learn-0.24.2-cp38-cp38-win_amd64.whl (6.9 MB)
  Using cached seaborn-0.11.1-py3-none-any.whl (285 kB)
Processing c:\users\joanp\appdata\local\pip\cache\wheels\0d\68\bb\926065fb744e7d7cb67334cb1a9c696722abc8303e5dc9a8d0\empyrical-0.5.5-py3-none-any.whl
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Using cached pandas_datareader-0.9.0-py3-none-any.whl (107 kB)
  Created wheel for pyfolio: filename=pyfolio-0.9.2-py3-none-any.whl size=88684 sha256=c2ab8be715e4b1e08146618404849394acced75beb53ac0ae8f6c145fc1f3dab
  Stored in directory: c:\users\joanp\appdata\local\pip\cache\wheels\42\16\92\9c9d49efccc497040268960f0825101cc9b1c6e41cdc5a409e
Successfully built pyfolio


<a id='1.2'></a>
## 2.2. Import Packages

In [81]:
import pandas as pd
from config import config
from dataset.download_dataset.cryptodownloader_binance import CryptoDownloader_binance
from preprocessing.preprocessors import FeatureEngineer
from preprocessing.data import data_split
from env.env_portfolio import StockPortfolioEnv
from model.models import DRLAgent
# from trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts


<a id='1.3'></a>
## 2.3 Create Folders

In [12]:
import os
download_data = False
if not os.path.exists(config.DATA_SAVE_DIR):
    os.makedirs(config.DATA_SAVE_DIR)
    download_data = True
if not os.path.exists(config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists(config.TENSORBOARD_LOG_DIR):
    os.makedirs(config.TENSORBOARD_LOG_DIR)
if not os.path.exists(config.RESULTS_DIR):
    os.makedirs(config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data

In [13]:
data_downloader = CryptoDownloader_binance(config.START_DATE, config.END_DATE, config.MULTIPLE_TICKER_8, config.DATA_SAVE_DIR, config.DATA_GRANULARITY)
if download_data:    
    data_downloader.download_data()
df = data_downloader.load()

In [14]:
df

,date,open,high,low,close,volume,tic
0,2020-01-01 00:00:00,7195.24,7196.25,7175.46,7177.02,511.814901,btc
1,2020-01-01 01:00:00,7176.47,7230.00,7175.71,7216.27,883.052603,btc
2,2020-01-01 02:00:00,7215.52,7244.87,7211.41,7242.85,655.156809,btc
3,2020-01-01 03:00:00,7242.66,7245.00,7220.00,7225.01,783.724867,btc
4,2020-01-01 04:00:00,7225.00,7230.00,7215.03,7217.27,467.812578,btc
...,...,...,...,...,...,...,...
87547,2021-03-31 19:00:00,193.98,195.25,192.35,193.05,38115.533510,ltc
87548,2021-03-31 20:00:00,193.03,196.39,192.48,195.66,32785.188130,ltc
87549,2021-03-31 21:00:00,195.65,196.90,194.63,195.29,20567.010810,ltc
87550,2021-03-31 22:00:00,195.31,197.00,195.23,195.80,8967.744990,ltc


# Part 4: Preprocess Data

In [15]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [16]:
df

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2020-01-01 00:00:00,0.032850,0.032850,0.032700,0.032780,1.166001e+06,ada,0.000000,0.033182,0.032588,100.000000,66.666667,100.000000,0.032780,0.032780
10944,2020-01-01 00:00:00,13.715900,13.721100,13.690300,13.698100,6.201669e+04,bnb,0.000000,0.033182,0.032588,100.000000,66.666667,100.000000,13.698100,13.698100
21888,2020-01-01 00:00:00,7195.240000,7196.250000,7175.460000,7177.020000,5.118149e+02,btc,0.000000,0.033182,0.032588,100.000000,66.666667,100.000000,7177.020000,7177.020000
32832,2020-01-01 00:00:00,0.002014,0.002023,0.002008,0.002008,9.630910e+05,doge,0.000000,0.033182,0.032588,100.000000,66.666667,100.000000,0.002008,0.002008
43776,2020-01-01 00:00:00,129.160000,129.190000,128.680000,128.870000,7.769173e+03,eth,0.000000,0.033182,0.032588,100.000000,66.666667,100.000000,128.870000,128.870000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43775,2021-03-31 23:00:00,0.053143,0.053911,0.053000,0.053770,4.448111e+07,doge,-0.000103,0.054076,0.052762,50.006619,-10.311768,28.977404,0.053646,0.053938
54719,2021-03-31 23:00:00,1903.970000,1924.210000,1901.620000,1919.370000,2.122478e+04,eth,26.214167,1948.647647,1762.109353,64.618395,164.610885,23.659415,1851.563000,1833.205500
65663,2021-03-31 23:00:00,28.571700,29.440100,28.550100,29.416700,4.557268e+05,link,0.276832,29.035767,26.359743,64.396467,226.662894,36.167462,27.746403,27.922328
76607,2021-03-31 23:00:00,194.620000,197.630000,194.470000,196.700000,2.863109e+04,ltc,0.332601,198.176985,188.501015,55.327678,76.382622,7.269085,194.139000,194.302667


## Add covariance matrix as states

In [17]:
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  covs = return_lookback.cov().values 
  cov_list.append(covs)
  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
        

In [ ]:
df

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2020-01-01 to 2020-12-31

In [22]:
train = data_split(df, '2020-01-01','2020-12-31')

## Environment for Portfolio Allocation


In [23]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()

Stock Dimension: 8, State Space: 8


<a id='5'></a>
# Part 6: Implement DRL Algorithms

In [24]:
# initialize
agent = DRLAgent(env = env_train)

### Model 1: **A2C**


In [33]:
A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 1, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device


In [34]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=60000)

  time_elapsed       | 324      |
|    total_timesteps    | 56500    |
| train/                |          |
|    entropy_loss       | -2.42    |
|    explained_variance | nan      |
|    learning_rate      | 0.0002   |
|    n_updates          | 56499    |
|    policy_loss        | 5.7e+06  |
|    std                | 0.331    |
|    value_loss         | 4.47e+12 |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 173       |
|    iterations         | 56600     |
|    time_elapsed       | 325       |
|    total_timesteps    | 56600     |
| train/                |           |
|    entropy_loss       | -2.41     |
|    explained_variance | nan       |
|    learning_rate      | 0.0002    |
|    n_updates          | 56599     |
|    policy_loss        | -2.43e+05 |
|    std                | 0.33      |
|    value_loss         | 4.69e+12  |
-------------------------------------
--------------------------

### Model 2: **PPO**

In [36]:
# agent = DRLAgent(env = env_train)
# PPO_PARAMS = {
#     "n_steps": 2048,
#     "ent_coef": 0.005,
#     "learning_rate": 0.0001,
#     "batch_size": 128,
# }
# model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [35]:
# trained_ppo = agent.train_model(model=model_ppo, 
#                              tb_log_name='ppo',
#                              total_timesteps=80000)

## Trading
Assume that we have $1,000,000 initial capital at 2021-01-01.

In [37]:
trade = data_split(df,'2021-01-01', config.END_DATE)
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [38]:
trade.shape

(17280, 16)

In [39]:
trade

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list
0,2021-01-01 00:00:00,0.181340,0.181460,0.178310,0.180510,1.919492e+07,ada,-0.000153,0.184657,0.176516,50.750459,-27.165052,6.100618,0.180782,0.182191,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,37.359600,37.442300,36.963600,37.376400,9.511383e+04,bnb,-0.083237,37.946399,36.719461,51.231566,-33.515795,11.715168,37.421230,37.590240,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,28923.630000,29031.340000,28690.170000,28995.130000,2.311811e+03,btc,133.938215,29333.932277,28372.364723,57.001762,31.346237,5.617701,28857.221667,28217.099500,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,0.004672,0.004701,0.004601,0.004679,2.768207e+07,doge,0.000018,0.004729,0.004558,53.910140,48.169530,6.372370,0.004639,0.004574,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,736.420000,739.000000,729.330000,734.070000,2.793270e+04,eth,-0.496311,752.068407,727.317593,50.347614,-92.128309,16.290326,741.657333,735.789500,"[[0.00023491493831790994, 7.732551281244585e-0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,2021-03-31 23:00:00,0.053143,0.053911,0.053000,0.053770,4.448111e+07,doge,-0.000103,0.054076,0.052762,50.006619,-10.311768,28.977404,0.053646,0.053938,"[[0.0001184825404164931, 7.097453732794227e-05..."
2159,2021-03-31 23:00:00,1903.970000,1924.210000,1901.620000,1919.370000,2.122478e+04,eth,26.214167,1948.647647,1762.109353,64.618395,164.610885,23.659415,1851.563000,1833.205500,"[[0.0001184825404164931, 7.097453732794227e-05..."
2159,2021-03-31 23:00:00,28.571700,29.440100,28.550100,29.416700,4.557268e+05,link,0.276832,29.035767,26.359743,64.396467,226.662894,36.167462,27.746403,27.922328,"[[0.0001184825404164931, 7.097453732794227e-05..."
2159,2021-03-31 23:00:00,194.620000,197.630000,194.470000,196.700000,2.863109e+04,ltc,0.332601,198.176985,188.501015,55.327678,76.382622,7.269085,194.139000,194.302667,"[[0.0001184825404164931, 7.097453732794227e-05..."


In [45]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

begin_total_asset:1000000
end_total_asset:4335102.482924683
Sharpe:  0.9835002656567612
hit end!


In [82]:
df_daily_return

,date,daily_return
0,2021-01-01 00:00:00,0.000000
1,2021-01-01 01:00:00,0.019965
2,2021-01-01 02:00:00,0.001651
3,2021-01-01 03:00:00,0.003178
4,2021-01-01 04:00:00,0.004175
...,...,...
2155,2021-03-31 19:00:00,-0.006622
2156,2021-03-31 20:00:00,0.011147
2157,2021-03-31 21:00:00,-0.000594
2158,2021-03-31 22:00:00,-0.000453


In [70]:
df_actions

,ada,bnb,btc,doge,eth,link,ltc,xrp
date,,,,,,,,
2021-01-01 00:00:00,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000
2021-01-01 01:00:00,0.146660,0.146660,0.146660,0.053953,0.146660,0.085913,0.126834,0.146660
2021-01-01 02:00:00,0.150024,0.150024,0.150024,0.055191,0.150024,0.106450,0.088240,0.150024
2021-01-01 03:00:00,0.152738,0.152738,0.152738,0.056189,0.152738,0.077550,0.102571,0.152738
2021-01-01 04:00:00,0.155483,0.155483,0.155483,0.057199,0.155483,0.070874,0.094512,0.155483
...,...,...,...,...,...,...,...,...
2021-03-31 19:00:00,0.150382,0.150382,0.150382,0.055322,0.150382,0.137445,0.055322,0.150382
2021-03-31 20:00:00,0.159565,0.159565,0.159565,0.058701,0.159565,0.074393,0.069079,0.159565
2021-03-31 21:00:00,0.154691,0.154691,0.154691,0.056908,0.154691,0.093923,0.075715,0.154691


In [68]:
df_actions.to_csv('df_actions.csv')

<a id='6'></a>
# Part 7: Backtest

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [78]:
# from pyfolio import timeseries
# DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
# perf_func = timeseries.perf_stats 
# perf_stats_all = perf_func( returns=DRL_strat, 
#                               factor_returns=DRL_strat, 
#                                 positions=None, transactions=None, turnover_denom="AGB")

NameError: name 'convert_daily_return_to_pyfolio_ts' is not defined

In [ ]:
# print("==============DRL Strategy Stats===========")
# perf_stats_all

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
# print("==============Compare to IHSG===========")
# %matplotlib inline
# BackTestPlot(df_account_value, 
#              baseline_ticker = '^JKSE', 
#              baseline_start = df_account_value.loc[0,'date'],
#              baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])